In [1]:
import numpy as np
import torch

import inspect
import os
import sys

sys.path.insert(0, os.path.abspath('../examples'))
import data

DEVICE = 'cuda:0' if torch.cuda.is_available() else 'cpu'
BASE_FOLDER = os.path.abspath('')
DATA_FOLDER = os.path.join(BASE_FOLDER, '../data')

### Data Setup

In [2]:
# Parameters
dataset_name = 'ISS'
dimensions = 16
seed = 42

# Derivatives
RUN_FOLDER = os.path.join(BASE_FOLDER, 'runs', dataset_name)

In [3]:
# Load data and save to file
modalities, types, features = data.load_data(dataset_name, DATA_FOLDER)

if not os.path.exists(RUN_FOLDER): os.mkdir(RUN_FOLDER)
for i in range(len(modalities)):
    modality = modalities[i]

    # Regular matrices (ManiNetCluster)
    np.savetxt(os.path.join(RUN_FOLDER, f'X{i+1}.txt'), modality, delimiter='\t')

    # Similarity matrices (MMD-MA)
    modality_z = modality - modality.mean(axis=0, keepdims=True) / modality.std(axis=0, keepdims=True)
    similarity = np.matmul(modality_z, modality_z.T)
    np.savetxt(os.path.join(RUN_FOLDER, f'X{i+1}_sim.tsv'), similarity, delimiter='\t')

### Methods
Arranged from fastest to slowest

In [4]:
# LMA
# https://github.com/namtk/ManiNetCluster/tree/master/inst/python
new_wd = os.path.join(RUN_FOLDER, 'LMA')
if not os.path.exists(new_wd): os.mkdir(new_wd)
os.chdir(new_wd)

!conda run -n maninetcluster python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} {os.path.join(RUN_FOLDER, 'X1.txt')} {os.path.join(RUN_FOLDER, 'X2.txt')} --align lma -p {dimensions}

os.chdir(BASE_FOLDER)

In [6]:
# CCA
# https://github.com/namtk/ManiNetCluster/tree/master/inst/python
new_wd = os.path.join(RUN_FOLDER, 'CCA')
if not os.path.exists(new_wd): os.mkdir(new_wd)
os.chdir(new_wd)

!conda run -n maninetcluster python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} {os.path.join(RUN_FOLDER, 'X1.txt')} {os.path.join(RUN_FOLDER, 'X2.txt')} --align cca -p {dimensions}

os.chdir(BASE_FOLDER)

In [5]:
# NLMA
# https://github.com/namtk/ManiNetCluster/tree/master/inst/python
new_wd = os.path.join(RUN_FOLDER, 'NLMA')
if not os.path.exists(new_wd): os.mkdir(new_wd)
os.chdir(new_wd)

!conda run -n maninetcluster python {os.path.join(BASE_FOLDER, 'maninetcluster_helper.py')} {os.path.join(RUN_FOLDER, 'X1.txt')} {os.path.join(RUN_FOLDER, 'X2.txt')} --align nlma -p {dimensions}

os.chdir(BASE_FOLDER)

In [7]:
# MMD-MA
# https://bitbucket.org/noblelab/2019_mmd_wabi/src/master/manifoldAlignDistortionPen_mmd_multipleStarts.py
new_wd = os.path.join(RUN_FOLDER, 'MMD-MA')
if not os.path.exists(new_wd): os.mkdir(new_wd)
os.chdir(new_wd)

!conda run -n mmdma python {os.path.join(BASE_FOLDER, 'MMD-MA/manifoldAlignDistortionPen_mmd_multipleStarts.py')} {os.path.join(RUN_FOLDER, 'X1_sim.tsv')} {os.path.join(RUN_FOLDER, 'X2_sim.tsv')} --seed {seed} --p {dimensions}
!python {os.path.join(BASE_FOLDER, 'mmd_helper.py')}

os.chdir(BASE_FOLDER)

^C
Traceback (most recent call last):
  File "/home/thema/repos/inept/other_methods/mmd_helper.py", line 4, in <module>
    alpha = np.loadtxt('alpha_hat_42_10000.txt')  # Hard-coded
  File "/home/thema/miniconda3/envs/inept/lib/python3.10/site-packages/numpy/lib/npyio.py", line 1373, in loadtxt
    arr = _read(fname, dtype=dtype, comment=comment, delimiter=delimiter,
  File "/home/thema/miniconda3/envs/inept/lib/python3.10/site-packages/numpy/lib/npyio.py", line 992, in _read
    fh = np.lib._datasource.open(fname, 'rt', encoding=encoding)
  File "/home/thema/miniconda3/envs/inept/lib/python3.10/site-packages/numpy/lib/_datasource.py", line 193, in open
    return ds.open(path, mode, encoding=encoding, newline=newline)
  File "/home/thema/miniconda3/envs/inept/lib/python3.10/site-packages/numpy/lib/_datasource.py", line 533, in open
    raise FileNotFoundError(f"{path} not found.")
FileNotFoundError: alpha_hat_42_10000.txt not found.
